In [167]:
import numpy as np
import math

In [43]:
# 2.1: solve system of linear equations
from scipy.optimize import linprog

# linear inequalities: A * x <= b
def get_entire_point(A, b):
    n = len(A[0])
    bounds_for_coordinates = [(None, None) for i in range(n)]
    coefficients_x_i = [0 for i in range(n)]  
    
    def get_x_i_bounds(i):
        bounds = None
        coefficients_x_i[i] = 1
        res_min = linprog(coefficients_x_i,
                          A_ub=A,
                          b_ub=b,
                          bounds=bounds_for_coordinates)
        coefficients_x_i[i] = -1
        res_max = linprog(coefficients_x_i,
                          A_ub=A,
                          b_ub=b,
                          bounds=bounds_for_coordinates)
        coefficients_x_i[i] = 0
        min_x_i = res_min.fun
        max_x_i = -res_max.fun
        if res_min.status == 3 and res_max.status == 3:
            bounds = [-1, 1]
        elif res_min.status == 3:
            bounds = [max_x_i - 2, max_x_i]
        elif res_max.status == 3:
            bounds = [min_x_i, min_x_i + 2]
        elif res_min.status == 0 and res_max.status == 0:
            bounds = [min_x_i, max_x_i]
        return bounds
    
    entire_point = []
    for i in range(n):
        bounds = get_x_i_bounds(i)
        if bounds is None or bounds[0] == bounds[1]:
            return None
        x_i = (bounds[0] + bounds[1]) / 2
        entire_point.append(x_i)
        bounds_for_coordinates[i] = (x_i, x_i)
    return entire_point

#test
entire_pt = get_entire_point([[1, 1], [-1, -1]], [3, -2])
assert entire_pt == [0, 2.5] 
print(entire_pt)

entire_pt = get_entire_point([[-1, 0], [0, -1], [1, 1]], [0, 0, 1])
assert entire_pt == [0.5, 0.25] 
print(entire_pt)

entire_pt = get_entire_point([[1, 0], [-1, 0]], [0, 0])
assert entire_pt is None
print(entire_pt)

entire_pt = get_entire_point([[1, 0], [-1, 0]], [-2, 0])
assert entire_pt is None
print(entire_pt)

entire_pt = get_entire_point([[1, 0], [-1, 0]], [2, 0])
assert entire_pt == [1, 0]
print(entire_pt)

[0.0, 2.5]
[0.5, 0.25]
None
None
[1.0, 0.0]


In [173]:
#2.2 interior point method
# A * x <= b
def interior_point_method(Aq, bq, c, A, b, x_start = None):
    iterations_number = 50
    m = len(b)
    n = len(A[0])
    if x_start is None:
        x_start = np.array(np.matrix(get_entire_point(A, b)).T)
    print("x_start: ", x_start)
        
    def func(x):
        return (1 / 2 * np.transpose(x) @ Aq @ x - np.transpose(bq) @ x + c)[0][0]
    
    def f_grad(x):
        return Aq @ x - bq
    
    def f_hess(x):
        return Aq
    
    # A, b, v -- arrays/ndarrays
    def log_bar_grad(v):
        s = 0
        for i in range(len(b)):
            s += 1.0 / (b[i][0] - np.dot(A[i], v)[0]) * A[i].T
        return np.matrix(s).T

    def log_bar_hess(v):
        s = 0
        d = np.diag(np.array([1.0 / (b[i][0] - (A[i] @ v)[0]) ** 2 for i in range(m)]))
        return A.T @ d @ A

    def F(x):
        return (lambda t: t * func(x) - sum([np.log(b[i][0] - (A[i] @ x)[0]) for i in range(m)]))
    
    def grad_F(x):
        return (lambda t: t * f_grad(x) + log_bar_grad(x))
    
    def hess_F(x):
        return (lambda t: t * f_hess(x) + log_bar_hess(x))
    
    #x_0 = np.array([[1 / 2], [1 / 2]])
    #print("x_0: ", x_0)
    #print("func(x_0): ", func(x_0))
    #print("f_grad(x_0): ", f_grad(x_0))
    #print("f_hess(x_0): ", f_hess(x_0))
    #print("log_bar_grad(x_0): ", log_bar_grad(x_0))
    #print("log_bar_hess(x_0): ", log_bar_hess(x_0))
    #print("F(x_0, 1): ", F(x_0)(1))
    #print("grad_F(x_0, 1): ", grad_F(x_0)(2))
    #print("hess_F(x_0, 1): ", hess_F(x_0)(2))
    
    alpha = 1.5
    t = 1.5
    trajectory = []
    trajectory.append(x_start.copy())
    cur_x = x_start.copy()
    for i in range(iterations_number):
        grad = grad_F(cur_x)(t)
        inv_hess = np.linalg.inv(hess_F(cur_x)(t))
        cur_x = cur_x - inv_hess @ grad
        cur_x = np.array(cur_x)
        trajectory.append((cur_x.copy(), func(cur_x)))
        t = alpha * t
    return trajectory

In [176]:
# example from first task, n = 2
Aq = np.array([[4, -2], [-2, 2]])
print("Aq: ", Aq)
bq = np.array([[1], [2]])
print("bq: ", bq)
c = 0
A = np.array([[1, 0], [-1, 0], [0, 1], [0, -1]])
print("A: ", A)
b = np.array([[1], [1], [1], [1]])
print("b: ", b)
trajectory = interior_point_method(Aq, bq, c, A, b)
min_point = np.array([[3 / 4], [1]])
min_value = -2.125
print("---------results---------")
print("Calculated min_value: ", trajectory[-1][1])
print("Calculated min_point: ", trajectory[-1][0])
print("Expected min_value: ", min_value)
print("Expected min_point: ", min_point)
assert math.isclose(trajectory[-1][1], min_value)

Aq:  [[ 4 -2]
 [-2  2]]
bq:  [[1]
 [2]]
A:  [[ 1  0]
 [-1  0]
 [ 0  1]
 [ 0 -1]]
b:  [[1]
 [1]
 [1]
 [1]]
x_start:  [[0.]
 [0.]]
---------results---------
Calculated min_value:  -2.124999998625483
Calculated min_point:  [[0.75]
 [1.  ]]
Expected min_value:  -2.125
Expected min_point:  [[0.75]
 [1.  ]]


In [177]:
# example from first task, n = 3
Aq = np.array([[4, -2, 1], [-2, 2, 1.5], [1, 1.5, 2]])
print("Aq: ", Aq)
bq = np.array([[1], [0], [0]])
print("bq: ", bq)
c = 5
A = np.array([[1, 0, 0], [-1, 0, 0], [0, 1, 0], [0, -1, 0], [0, 0, 1], [0, 0, -1]])
print("A: ", A)
b = np.array([[1], [0], [1], [0], [1], [0]])
print("b: ", b)
trajectory = interior_point_method(Aq, bq, c, A, b)
min_point = np.array([[1 / 2], [1 / 2], [0]])
min_value = 4.75
print("Calculated min_value: ", trajectory[-1][1])
print("Calculated min_point: ", trajectory[-1][0])
print("Expected min_value: ", min_value)
print("Expected min_point: ", min_point)
assert math.isclose(trajectory[-1][1], min_value)

Aq:  [[ 4.  -2.   1. ]
 [-2.   2.   1.5]
 [ 1.   1.5  2. ]]
bq:  [[1]
 [0]
 [0]]
A:  [[ 1  0  0]
 [-1  0  0]
 [ 0  1  0]
 [ 0 -1  0]
 [ 0  0  1]
 [ 0  0 -1]]
b:  [[1]
 [0]
 [1]
 [0]
 [1]
 [0]]
x_start:  [[0.5]
 [0.5]
 [0.5]]
Calculated min_value:  4.750000001403333
Calculated min_point:  [[4.99999999e-01]
 [4.99999998e-01]
 [1.12266612e-09]]
